# Orbital motion
Now we will start on the problem that the Kalman filter was most famously first used for (and the context where I learned it): Orbital motion. The code in this notebook is mostly related to doing symbolic computation of the needed derivative matrices.

$$
\def\M#1{{[\mathbf{#1}]}}
\def\MM#1#2{{[\mathbf{#1}{#2}]}}
\def\E{\operatorname{E}}
\def\cov{\operatorname{cov}}
\def\T{^\mathsf{T}}
$$


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as nr

In [ ]:
%%capture
%run Covariance.ipynb

In [ ]:
%%capture
%run ExtendedKalman.ipynb

## Problem definition

This is the simplest form of the orbit determination problem I learned this filter on back in school. A satellite is in orbit around a point mass. Newton's gravitation applies. The satellite is tracked from a radar at a fixed point above the point mass (or on the surface of a perfectly smooth, homogeneous, radar-transparent planet) in the plane of the satellite orbit. The point mass is so much heavier than the satellite that we may treat the satellite as a test mass. This means that the gravity of the satellite does not significantly move the central mass, and therefore the mass of the satellite cancels right out of the problem.

## State Vector

This is like the cart, but now we have two dimensions. It is provable that the trajectory of any test mass around a point mass is confined to a plane containing the point mass, as long as it is only under the influence of gravity, or if all other forces are in the plane also.


So, our state vector has four components, two for position and two for velocity, for a total of four.

$$
\vec{x}=\begin{bmatrix}r_x \\ r_y \\ v_x \\ v_y\end{bmatrix}
$$

We are using $r$ instead of $p$ for position, mostly for historical reasons. Think of it as the "ray" or "radius" from the center of mass to the satellite. The names of the components suggest that we can have:

$$\begin{eqnarray*}
\vec{r}&=&\begin{bmatrix}r_x \\ r_y\end{bmatrix}\\
\vec{v}&=&\begin{bmatrix}v_x \\ v_y\end{bmatrix}\\
\end{eqnarray*}$$

We may use this notation when appropriate, meaning that the whole state is the concatenation of these two vectors:

$$
\vec{x}=\begin{bmatrix}\vec{r} \\ - \\ \vec{v}\end{bmatrix}
$$

## Physics

This time we use Newton's Universal Law of Gravitation (in title caps, because it is important.)

$$F=\frac{GMm}{r^2}$$

This is combined with Newton's second law:

$$F=ma$$

to get:

$$ma=\frac{GMm}{r^2}$$

where:

* $F$ is the magnitude of the force on the test mass (only used in this section, above this point, a few times, not to be confused with function $F(\vec{x})$ which is used to describe the rate of change of the state vector).
* $G$ is the universal gravitational constant
* $M$ is the mass of the central mass
* $m$ is the mass of the test mass, $m<<M$
* $r=\sqrt{r_x^2+r_y^2}$ is the distance from the central mass to the test mass
* $a$ is the magnitude of the acceleration imposed on the test mass

We see that m appears on both sides, so we will cancel it out and get:

$$a=\frac{GM}{r^2}$$

Also, this is a vector acceleration, directed towards the center of mass, so multiply this by a unit vector pointing from the test mass to the center $-\vec{r}/r$ to get

$$\vec{a}=\frac{-GM\vec{r}}{r^3}$$

It is very difficult to measure $G$ by itself, as this involves measuring gravity exerted by objects small enough to fit in a laboratory. It is also very difficult to measure $M$ of a planet by itself, because it's a whole planet, and won't fit on a scale. However, by observing the motions of satellites, it turns out that it is easy to accurately measure the product $GM$ much more accurately than either $G$ or $M$. So, we call this product $\mu$ (Greek small mu) and use it instead.

$$\begin{eqnarray*}
F(\vec{x})&=&\vec a \\
                &=&\frac{-\mu\vec r}{r^3}
                \end{eqnarray*}$$

And there's our physics function F(). If we want, we can expand it by element

$$\begin{eqnarray*}
F_{r_x}&=&v_x \\
F_{r_y}&=&v_y \\
F_{v_x}&=&-\frac{\mu r_x}{\left(\sqrt{r_x^2+r_y^2}\right)^3} \\
F_{v_y}&=&-\frac{\mu r_y}{\left(\sqrt{r_x^2+r_y^2}\right)^3} \\
\end{eqnarray*}$$

## Observation

Our radar is able to measure the distance (only) from itself to the satellite. Therefore the measurement vector at any time is just this distance, only one component again. The station is located at $\vec{m}=\begin{bmatrix}m_x & m_y\end{bmatrix}\T$, so the distance from the radar to the satellite is

$$\begin{eqnarray*}
g(\vec{x})&=&\left|\vec{r}-\vec{m}\right| \\
          &=&\sqrt{(r_x-m_x)^2+(r_y-m_y)^2}
\end{eqnarray*}$$

## Jacobian matrices

First, $\M \Phi$. This one has a lot of blank space because two of its components are simple variables, and the other two are functions of only two elements of the state vector, so right off we have:

$$
\M \Phi=\begin{bmatrix}0 & 0 & 1 & 0 \\
           0 & 0 & 0 & 1 \\
           \frac{dF_{v_x}}{dr_x} & \frac{dF_{v_x}}{dr_y} & 0 & 0 \\
           \frac{dF_{v_y}}{dr_x} & \frac{dF_{v_y}}{dr_y} & 0 & 0 \end{bmatrix}$$

Only six components out of 16 are nonzero, and only four are nontrivial. So, what about those?

$$
F_{v_x}=-\frac{\mu r_x}{\left(\sqrt{r_x^2+r_y^2}\right)^3}
$$

To the derivinator! [derivative of mu*x/(sqrt(x^2+y^2))^3](http://www.wolframalpha.com/input/?i=derivative+of+mu*x%2F%28sqrt%28x%5E2%2By%5E2%29%29%5E3) I don't think that Alpha is smart enough to substitute r, so I just put it in explicitly, and we can sub it back out when we get the result. Alpha (now I want to call it "Ziggy" for some reason) says that the result is


    (d)/(dx)((mu x)/sqrt(x^2+y^2)^3) = (mu (y^2-2 x^2))/(x^2+y^2)^(5/2)


or cleaning up and putting back in $r=\sqrt{r_x^2+r_y^2}$

$$
\frac{dF_{v_x}}{dr_x}=-\frac{\mu(r_y^2-2r_x^2)}{r^5}
$$

We continue to get the other three elements:

$$
\begin{eqnarray*}
\frac{dF_{v_x}}{dr_y}&=& -\frac{\mu(3r_x r_y)}{r^5} \\
\frac{dF_{v_y}}{dr_x}&=& -\frac{\mu(3r_x r_y)}{r^5} \\
\frac{dF_{v_y}}{dr_y}&=&-\frac{\mu(r_x^2-2r_y^2)}{r^5}
\end{eqnarray*}$$

Note that now that we have the power of Python and SymPy in our corner, we can do something like this:

In [ ]:
import sympy
from IPython.display import display
#Symbolic form of physics equations, so that they can be symbolically differentiated
rx,ry,vx,vy,mu=sympy.symbols('r_x r_y v_x v_y mu')
r=sympy.sqrt(rx**2+ry**2)
Frx=vx
Fry=vy
Fvx=-mu*rx/r**3
Fvy=-mu*ry/r**3

(F_B612,Phi_B612)=fjac([rx,ry,vx,vy],[mu],[Frx,Fry,Fvx,Fvy])
xh=np.array([[3,3],[5,5],[0,0],[0,0]])
m=[1000]
#Demonstrate multiplication of stacks of matrices
Phi=Phi_B612(xh,m)
print(Phi@Phi)

Now for $\M H$. This has four elements, one row for the one element of the observation by four columns for the elements of the state, but two of them are zero since the measurement doesn't involve the velocity elements of the state. We will define $\rho$ (Greek small rho) as the measurement itself, so:

$$\begin{eqnarray*}
\rho&=&g(\vec{x}) \\
    &=&\sqrt{(r_x-m_x)^2+(r_y-m_y)^2}
\end{eqnarray*}$$

$$\begin{eqnarray*}
\frac{dg}{dr_x}&=&\frac{r_x-m_x}{\rho} \\
\frac{dg}{dr_y}&=&\frac{r_y-m_y}{\rho} \\
\end{eqnarray*}$$


In [ ]:
mx,my=sympy.symbols('m_x m_y')
rho=sympy.sqrt((rx-mx)**2+(ry-my)**2)
ctheta=(rx-mx)/rho
stheta=(ry-my)/rho
(g_B612,H_B612)=fjac([rx,ry,vx,vy],[mx,my],[rho,ctheta,stheta])

m=[0,1]
print("xh=",xh)
print("g()=",g_B612(xh,m))
print("H()=",H_B612(xh,m))

# Worked example

We can hang this model on the same scaffold we built last time. Mostly we just change the model functions, but we are also going to change the simulator at the top, since this model is more complicated than last time.

We are going to orbit planet [B6](https://en.wikipedia.org/wiki/B_612_(The_Little_Prince))[12](http://en.wikipedia.org/wiki/46610_Besixdouze), a perfectly spherical, homogeneous, radar-transparent planet which has a radius of 10m but sufficient gravity to live on conveniently.  The planet has a surface gravity of about 1g, exactly 10m/s^2. 

$$\begin{eqnarray*}
a&=&\frac{\mu}{r^2} \\
ar^2&=&\mu \\
\end{eqnarray*}$$

In [ ]:
r_B612=10 #m
a0_B612=10 #m/s**2
mu_B612=a0_B612*r_B612**2
print(mu_B612)

In units of meters and seconds, it has $\mu=1000$, which means that a ball thrown at 10m/s at the surface will stay in a circular orbit right at the surface, and circle the planet in 6.283 seconds.

> As an aside, B612 is one weird planet. 

In [ ]:
G=6.67430e-11 #Universal gravitational constant, m**3/kg/s**2, CODATA 2018 value
M_B612=mu_B612/G
print("M=%e kg"%M_B612)
V_B612=4/3*np.pi*r_B612**3
rho_B612=M_B612/V_B612
print("V=%f m**3"%V_B612)
print("(water=1000kg/m**3) rho=%e kg/m**3"%rho_B612)

rho_granite=2700 #kg/m**3
comp=6
rho_granite_comp=rho_granite*(9**comp)
print("%d times compressed=%e kg/m**3"%(comp,rho_granite_comp))

> It needs to weigh almost 15 billion tons to exert this much gravity, and therefore have a density of more than 3 million times that of water -- white dwarf degenerate matter is about 1 million times as dense as water. The object is twice as dense as sextuple compressed cobblestone. 
> 
> And yet it's radar-transparent...

We use the physics and observation functions and Jacobians defined above, so most of what is below is driver code.

In this first bit, we call a function called ``trueState()`` which we pass in the initial conditions, the timestamps we care about, and the physics function. It then does all the integrating and returns a list of true state vectors at all the requested times.

While we could pass in the physics function (and this would be a good idea for an arbitrary physics function), we have an analytical solution, since this is the Kepler problem. 

In [ ]:
%%capture
%run UniversalVariables.ipynb

In [ ]:
def trueState(t,xt0,F,Fpar,nstep=1000):
    """
    Calculate the true trajectory, given the true initial condition and the physics function F
    """
    return kepler(xt0,None,t,l_DU=1,mu=Fpar[0])

    #xt=np.zeros((xt0.size,t.size)) #True state vector (to be calculated)
    #xt[:,0]=xt0.ravel()
    ##Do a numerical integrator to get the positions
    #for i in range(1,t.size):
    #    xt_im1=xt[:,i-1].reshape(-1,1)  #Previous true state
    #    xt_i=xt_im1       #Will contain current true state
    #    dt=t[i]-t[i-1]
    #    ddt=dt/nstep;         #Do substeps between measurements
    #    for j in range(1,nstep):
    #        #Just use our physics function - that's what it is there for.
    #        xt_i=xt_i+ddt*F(xt_i,Fpar)
    #    xt[:,i]=xt_i.ravel();
    #return xt

t=np.arange(0,10,0.1)
xt0=np.array([[11],[0],[0],[10]])
xt=trueState(t=t,xt0=xt0,F=F_B612,Fpar=[1000])
import matplotlib.pyplot as plt
plt.figure("True trajectory")
plt.plot(xt[0,:],xt[1,:])
plt.plot(r_B612*np.sin(np.arange(0,6.28,0.01)),r_B612*np.cos(np.arange(0,6.28,0.01)),'k-')
plt.axis('equal')
plt.show()

Next, we call trueObs(), which takes a list of state vectors and the observation function, and produces a list of observations, with noise sprinkled on it if requested. In this case, we call it twice, once with noise and once without, so we can compare the Truth to our estimate.

In [ ]:
def trueObs(xt,g,gpar,R=None,seed=3217):
    #Size the problem
    s=xt.shape
    n=s[0]
    xt0=xt[:,0].reshape(-1,1)
    z=np.zeros((g(xt0,m).size,s[1]))
    #Clean observations
    for i in range(s[1]):
        z[:,i]=g(xt[:,i].reshape(n,1),gpar).ravel()
  
    #We want random noise, but the *same* random noise each time we run
    if R is not None:
        if seed is not None:
            nr.randn(seed)
        pn=pnoise(s[1],R) #Measurement noise
        z=z+pn            #Noisy Observations
    return z

m=[10,0]
zt=trueObs(xt,g_B612,m)
R=np.array([[0.5**2]])
zn=trueObs(xt,g_B612,m,R)
plt.figure("Observations")
plt.plot(t,zt[0,:],'-')
plt.plot(t,zn[0,:],'.')
plt.show()

This last part is the driver, and as we can see, most of the code has been farmed out to various subfunctions defined above.

In [ ]:
def nonlinear_B612(Q,R,t1=20,tag=""):
    # Simulated actual trajectory. Unperturbed orbit around B612 at slightly
    # higher than circlular velocity.

    t = np.arange(0, t1, 0.1)  # Timestamp of each measurement
    xt0 = np.array([[11], [0], [0], [10]])  # Position <11,0> meters from center, velocity <0,10>
    mu = [1000]
    xt = trueState(t, xt0, F_B612, mu)

    m = [10, 0]
    zt = trueObs(xt, g_B612, m)
    print(R)
    z = trueObs(xt, g_B612, m, R)
    #z=zt #Give it perfect measurements, so there should be zero residual

    [xh, P] = ekf_loop(xh_0=np.array([[12], [0], [0], [9]]),  # Initial state vector guess, slightly wrong
                       P_0=np.diag([1, 1, 1, 1]),  # Initial state vector uncertainty, 1m in position and 1m/s in velocity
                       z=z, t=t, F=F_B612, Phi=Phi_B612, Fpar=mu, g=g_B612, H=H_B612,gpar=m,Q=Q,R=R)

    # Plot some interesting results
    plt.figure("Nonlinear trajectory"+tag)
    plt.plot(10 * np.cos(np.arange(0, 2 * np.pi, 0.01)), 10 * np.sin(np.arange(0, 2 * np.pi, 0.01)), 'k-',label='Planet B612 surface')
    plt.plot(xt[0, :], xt[1, :], 'r+-',label='True position')
    plt.plot(xh[0, :], xh[1, :], 'b+',label='Estimated position')
    plt.legend()
    plt.axis('equal')

    # measurement residual plot
    zh = trueObs(xh, g_B612, m);
    plt.figure("Nonlinear measurements"+tag)
    plt.plot(t, zt[0,:], 'r-',label='Actual radar distance')
    plt.plot(t, zh[0,:], 'b+',label='Radar distance from estimate');
    plt.legend()

    plt.figure("Nonlinear residuals"+tag)
    plt.plot(t, zt[0,:]-zh[0,:],'+', label='residuals')
    plt.plot(t, zt[0,:]-z[0,:],'+', label='measurement noise')
    plt.plot(t, t*0, 'k-',label='residual=0');
    plt.legend()
    plt.show()
    # legend(,);


Let's see how we did. First, use 10cm noise and a nonphysical Q with 10cm noise on velocity per step:


In [ ]:
Q = np.diag([0, 0, 0.01, 0.01])  # Non-physical process noise covariance
R = np.array([[0.1 ** 2]])  # Measurement noise covariance
nonlinear_B612(Q,R,t1=10)

This is interesting. The filter is satisfied it did a good job -- the estimated trajectory has residuals well within the expected range, given the amount of noise we put on the measurements. However, it does this by taking advantage of the process noise to fudge the trajectory around quite a bit. Since there is no real process noise in this case, let's try with a zero $\M Q$:

In [ ]:
Q = np.diag([0, 0, 0, 0])  # Process noise covariance, 0 position and 10cm/s^2 on velocity
#R = np.array([[0**2]])  # Measurement noise covariance
nonlinear_B612(Q,R,t1=10,tag=" zero Q, zero R")

It has a rough start, and never really settles in well. Maybe the initial estimate is just too far off. In any case, I think one thing going on here is that the measurement we have doesn't constrain the orbit, IE the state isn't observable from this measurement type.